# Pictures acquisition

In [ ]:
import cv2 as cv
import os
import re

OpenCV fornisce una serie di modelli pre-addestrati per la detection di specifiche classi, basati sul metodo di Viola e Jones:<br/>
https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html<br/>
https://docs.opencv.org/3.4/dc/d88/tutorial_traincascade.html

Nel caso di installazione basata su *conda*, questi modelli possono essere trovati in:<br/>
``.conda/envs/<ambiente>/Lib/site-packages/cv2/data/``

Nel caso di installazione basata su *virtualenv*, questi modelli possono essere trovati in:<br/>
``<ambiente>/lib/python<X>/site-packages/cv2/data/``

In [ ]:
# Caricamento modello per il rilevamento di volti frontali
model_filename = 'C:/Users/gianc/Anaconda3/envs/dsim/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml'
face_detector = cv.CascadeClassifier(model_filename)

`cv.CascadeClassifier.detectMultiScale(image[, scaleFactor[, minNeighbors[, flags[, minSize[, maxSize]]]]]`

Parametri aggiuntivi:

- https://docs.opencv.org/2.4/modules/objdetect/doc/cascade_classification.html
- https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html
- https://docs.opencv.org/3.4.7/dc/da5/tutorial_py_drawing_functions.html

In [ ]:
def crop_picture(img, more_space=30):
    m = more_space # some additional space around the cropped face
    if img is not None:
        faces = face_detector.detectMultiScale(img[:,:,1]) # (img_gray)
        if len(faces) > 0:
            (x,y,w,h) = faces[0]
            img = img[(y-m):(y+h+m), (x-m):(x+w+m), :]
    return img

In [ ]:
def save_pictures(name, basedir, file_format, crop=True, more_space = 30):
    name_basedir = basedir + "/" + name + "/"
    if not os.path.isdir(name_basedir):
        print(f"Created new directory {name_basedir}")
        os.makedirs(name_basedir)

    current_images = [img for img in os.listdir(name_basedir) if img.endswith(file_format)]
    
    # latest picture will be the id of the most recent picture in the 'name_basedir' directory
    # so this function will note overwrite old pictures
    # if the "name_basedir" directory is empty, it will start from 0
    if len(current_images) == 0:
        latest_picture = -1
    else:
        latest_picture = max([int(re.findall("\d+", current_images[i])[0]) for i in range(len(current_images))])
        # alternative
        # latest_picture = max([int(current_images[i].split("_")[1].split(".")[0]) for i in range(len(current_images))])
    
    filename_format = f"{basedir}/{name}/{name}_"+"{}"+f".{file_format}"
    
    i = latest_picture
    cap = cv.VideoCapture(0)
    while(True):
        
        r, frame = cap.read()
        if crop:
            frame = crop_picture(frame, more_space)
            
        # Visualizzazione esterna
        cv.imshow('Video', frame)
        
        # Save is 's' is pressed
        if cv.waitKey(1) & 0xFF == ord('s'):
            
            # If crop is set to true, to make sure that the image has been cropped we check if it is a square
            if frame.shape[0] == frame.shape[1] or (not crop):
                
                i += 1
                filename = filename_format.format(i)
                cv.imwrite(filename, frame)
                print(f"Saved {filename}".replace("//", "/"))
                
        # exit if 'q' is pressed
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv.destroyAllWindows()

In [ ]:
name = "gian"
basedir = "./pictures/"
file_format = "png" # or jpg?

In [ ]:
save_pictures(name, basedir, file_format)

It works, but sometimes you need to spam the "s" or "q" keys...

# Move pictures to train/val/test folders

In [ ]:
import os
import numpy as np
import shutil
import random

In [ ]:
# # Creating Train / Val / Test folders (One time use)
root_dir = 'pictures/'
dest_dir = 'pictures2/' # or the same as root_dir
classes_dir = ['gian', 'cami']

val_ratio = 0.25
test_ratio = 0.15

In [ ]:
np.random.seed(1)

In [ ]:
for cls in classes_dir:
    os.makedirs(dest_dir +'train/' + cls, exist_ok=True)
    os.makedirs(dest_dir +'val/' + cls, exist_ok=True)
    os.makedirs(dest_dir +'test/' + cls, exist_ok=True)


    # Creating partitions of the data after shuffeling
    src = root_dir + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - val_ratio - test_ratio)), 
                                                               int(len(allFileNames)* (1 - test_ratio))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print(f'Total images: {len(allFileNames)}')
    print(f'Training: {len(train_FileNames)}')
    print(f'Validation: {len(val_FileNames)}')
    print(f'Testing: {len(test_FileNames)}\n')
    

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, dest_dir +'train/' + cls)

    for name in val_FileNames:
        shutil.copy(name, dest_dir +'val/' + cls)

    for name in test_FileNames:
        shutil.copy(name, dest_dir +'test/' + cls)